In [10]:
import pandas as pd
from tqdm import tqdm
import glob
import os

path = %pwd
while "src" in path:
    %cd ..
    path = %pwd


In [11]:
train_data_path = "data/train"
val_data_path = "data/val"
processed_data_path = "data/processed"
ast_folder_name = "ast"
concept_folder_name = "concept"
rel_folder_name = "rel"
txt_folder_name = "txt"

In [12]:
text_files = glob.glob(train_data_path + os.sep + txt_folder_name + os.sep +  "*.txt")

In [13]:
%pip install dateparser

  Using cached dateparser-1.1.0-py2.py3-none-any.whl (288 kB)
  Using cached tzlocal-4.1-py3-none-any.whl (19 kB)
  Using cached pytz_deprecation_shim-0.1.0.post0-py2.py3-none-any.whl (15 kB)
  Using cached tzdata-2021.5-py2.py3-none-any.whl (339 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
from dateparser import parse

results = pd.DataFrame()
Patient_id = [] 
Age = []
Sexe = []
Birthdate = []
Admission_date = []
Discharge_date = []
for file in tqdm(text_files):
    with open(file, "r") as f:
        lines = f.readlines()
        # remove endlines
        lines = [line.strip() for line in lines]
        # remove empty lines
        lines = [line.lower() for line in lines if line.strip()]

        try :
            age = lines[lines.index("age :")+ 1][0]
            if age[0].isdigit():
                # keep only digits
                age = "".join([c for c in age if c.isdigit()])
                Age.append(age)
            else:
                Age.append(None)
        except :
            Age += [None]
        
        try :
            sx = lines[lines.index("sex :")+ 1]
            if sx[0] in ['m', 'f']:
                Sexe.append('M' if sx[0] == "m" else 'F')
            else:
                Sexe.append("N/A")    
        except :
            Sexe += ["N/A"]
        
        
        try :
            if lines[lines.index("date of birth :")+ 1][0].isdigit():
                date= parse(lines[lines.index("date of birth :")+ 1]).strftime("%Y/%m/%d")
                Birthdate.append(date)
            else:
                Birthdate.append(None)
        except :
            Birthdate.append(None)
        
        try :
            if lines[lines.index("admission date :")+ 1][0].isdigit():
                date = parse(lines[lines.index("admission date :")+ 1]).strftime("%Y/%m/%d")
                Admission_date.append(date)
            else:
                Admission_date.append(None)
        except :
            Admission_date.append(None)
   
        
        try :
            if lines[lines.index("discharge date :")+ 1][0].isdigit():
                date = parse(lines[lines.index("discharge date :")+ 1]).strftime("%Y/%m/%d")
                Discharge_date.append(date)
            else:
                Discharge_date.append(None)
        except :
            Discharge_date.append(None)

results["filename"] = [f.split(os.sep)[-1][:-4] for f in text_files]
results["age"] = Age
results["sexe"] = Sexe
results["birthdate"] = Birthdate
results["admission_date"] = Admission_date
results["discharge_date"] = Discharge_date

print(results.isna().sum())

results

  0%|          | 0/170 [00:00<?, ?it/s]/home/mus5900/anaconda3/envs/nlp/lib/python3.9/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
100%|██████████| 170/170 [00:17<00:00,  9.72it/s]


filename            0
age               137
sexe                0
birthdate         102
admission_date     41
discharge_date     44
dtype: int64


,filename,age,sexe,birthdate,admission_date,discharge_date
0,018636330_DH,None,N/A,None,2005/06/02,2005/06/05
1,026350193_RWH,5,N/A,None,None,None
2,037945397_RWH,7,N/A,None,None,None
3,044687343_ELMVH,None,N/A,None,2006/03/13,2006/03/19
4,060376519_DH,4,N/A,None,None,None
...,...,...,...,...,...,...
165,record-80,None,M,2017/09/22,2017/09/22,2017/09/27
166,record-81,None,M,None,2011/02/06,2011/02/08
167,record-82,None,N/A,None,2015/03/16,2015/03/19
168,record-83,None,F,1930/12/19,2013/11/04,2013/11/16


In [15]:
# results
# save as jsonl
os.makedirs("data/train/metadata", exist_ok=True)
files = glob.glob("data/train/metadata/*")
for file in files:
    os.remove(file)
    
for i in range(len(results)):
    filename = results.loc[i, "filename"]
    results.iloc[i].to_json(f"data/train/metadata/{filename}.json")
